In [4]:
import os
import findspark
findspark.init(os.getenv('SPARK_HOME'))
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/28 11:39:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
train, test = data.randomSplit([0.8, 0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [10]:
model = als.fit(train)

In [12]:
predictions = model.transform(test)
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      2|   1.0|    12|   0.9086098|
|      7|   3.0|    12|   1.0254602|
|      0|   1.0|    22|  0.15354495|
|      3|   1.0|     1|    0.408868|
|      5|   1.0|    13|   0.6600409|
|      9|   1.0|    16|   1.6326547|
|      7|   3.0|     3|  -0.6575712|
|      1|   1.0|    20|-0.018660307|
|      6|   1.0|    20|   1.9401929|
|      0|   1.0|     5|   0.6995617|
|      8|   1.0|     5|   -1.817919|
|      0|   1.0|    19|  0.48305404|
|      9|   1.0|     9|   3.6664171|
|      0|   1.0|     8|  0.98978734|
|      0|   1.0|    23|   1.6877067|
|      1|   1.0|     7| -0.67230934|
|      4|   1.0|    24|    2.669012|
|      0|   1.0|    11|   3.5645177|
|      9|   1.0|    14|  -1.5549853|
|      5|   2.0|    18|    1.830044|
+-------+------+------+------------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse_eval = evaluator.evaluate(predictions)

In [14]:
rmse_eval

1.806499427805494

In [18]:
user = test.filter(test['userId'] == 11).select('movieId', 'userId')
user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     16|    11|
|     19|    11|
|     32|    11|
|     47|    11|
|     50|    11|
|     76|    11|
|     78|    11|
|     79|    11|
|     89|    11|
+-------+------+



In [22]:
recommendations = model.transform(user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|      0|    11| 3.5645177|
|     89|    11|  3.424339|
|     76|    11| 2.6862948|
|     16|    11| 2.6703348|
|     78|    11| 1.2480321|
|     19|    11| 1.0203769|
|     50|    11| 0.9146455|
|     79|    11|0.62460893|
|     47|    11|0.02556789|
|     32|    11|-1.3498268|
+-------+------+----------+

